In [ ]:
# figure02: measure likelihood for shared and independent covariance matrix
# author: Amir Farzmahdi
# last update: May 29th, 2024

In [ ]:
# required packages
import math
import pickle
import random
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import scipy as sp

In [ ]:
# set random seed for NumPy and Python's random module
random.seed(42)
np.random.seed(42)

In [ ]:
# settings
n_loc = 81
n_theta = 9
ndim = 10000  # number of test images

nfilt = 36
ncs_s = 36
ncs_i = 36

In [ ]:
with open('train_test_res.csv', "rb") as fp:
    train_test_res = pickle.load(fp)  

train_cov_mat = train_test_res['train_cov_mat']
test_filter_sz_res_conds = train_test_res['test_filter_sz_res_conds']

In [ ]:
# selected image index
img_idx = np.arange(0,ndim)

In [ ]:
# select response of large images to measure the likelihood
test_filter_sz_res = np.squeeze(test_filter_sz_res_conds[1,:,img_idx,:])
print('shape of test data (image x pair x filter):',test_filter_sz_res.shape)
n_timgs = test_filter_sz_res.shape[0]

In [ ]:
# geometry mean function
def geo_mean_overflow(iterable):
    return np.exp(np.log(iterable).mean())

In [ ]:
# generate a mask for cov matrix of independent case
mask = np.zeros((nfilt,nfilt))
mask[:int(nfilt/2),:int(nfilt/2)] = 1
mask[int(nfilt/2):,int(nfilt/2):] = 1

In [ ]:
# plot the mask
fig = plt.figure(1,figsize=(3.5,3))
cax = sns.heatmap(mask)
cbar = cax.collections[0].colorbar

# # set axes ticks
xticks = [0,18,36]
xticks_offest = 0
xticks = [x + xticks_offest for x in xticks]

yticks = [0,18,36]
yticks_offest = 0
yticks = [x + yticks_offest for x in yticks]

xtick_labels = np.char.mod('%d', xticks)
ytick_labels = np.char.mod('%d', yticks)

plt.xticks(xticks,labels=xtick_labels)
plt.yticks(yticks,labels=ytick_labels)
plt.tick_params(labelsize=8, width=1, length=0.5,
                direction='out',which='major',
                right=False,top=False,bottom=True,left=True)
# set labels
plt.xlabel('filter index')
plt.ylabel('filter index')

# set colorbar parameters
cbar.set_ticks([0, 0.5, 1])
cbar.set_ticklabels(['0', '0.5', '1'])
cbar.ax.tick_params(labelsize=8, width=1, length=0.5)

In [ ]:
# initialize arrays to store likelihoods, based on Coen-Cagli et. al (2009)
p_shared_locs_imgs = np.zeros((n_loc, n_theta))
p_indp_locs_imgs = np.zeros((n_loc, n_theta))
p_diff = np.zeros((n_loc, n_theta))

# loop over locations and orientations
for k in range(n_loc):
    for j in range(n_theta):
        idx = np.linspace((j * nfilt), ((j + 1) * nfilt - 1), nfilt).astype(int)
        
        # shared case
        cov_mat_shared = train_cov_mat[k, j, :, :]
        inv_cov_mat_shared = np.linalg.inv(cov_mat_shared)
        det_inv_cov_mat_shared = np.linalg.det(inv_cov_mat_shared)
        
        # independent case
        cov_mat_indp = cov_mat_shared * mask
        inv_cov_mat_indp = np.linalg.inv(cov_mat_indp)
        det_inv_cov_mat_indp = np.linalg.det(inv_cov_mat_indp)
        
        p_shared = []
        p_indp = []
        
        # loop over test images
        for i in range(n_timgs):
            img_res = test_filter_sz_res[i, k, idx]
            
            # shared case likelihood
            lambda_shared = np.sqrt(img_res.T @ inv_cov_mat_shared @ img_res)
            p_shared.append((np.power(det_inv_cov_mat_shared, 1/2) / np.power(2*math.pi, ncs_s/2)) *
                            (sp.special.kv((1-ncs_s/2), lambda_shared) / np.power(lambda_shared, ncs_s/2-1)))
            
            # independent case likelihood
            lambda_indp = np.sqrt(img_res.T @ inv_cov_mat_indp @ img_res)
            p_indp.append((np.power(det_inv_cov_mat_indp, 1/2) / np.power(2*math.pi, ncs_i/2)) *
                          (sp.special.kv((1-ncs_i/2), lambda_indp) / np.power(lambda_indp, ncs_i/2-1)))
        
        # calculate geometric mean of likelihoods
        p_shared_locs_imgs[k, j] = np.exp(np.mean(np.log(p_shared))) if p_shared else 0.0
        p_indp_locs_imgs[k, j] = np.exp(np.mean(np.log(p_indp))) if p_indp else 0.0
        p_diff[k, j] = np.log(p_shared_locs_imgs[k, j]) - np.log(p_indp_locs_imgs[k, j])

In [ ]:
# save natural images likelihood of shared versus independent pairs
save_file_path = "p_shared_ind_nat_test_images.csv"
with open(save_file_path, "wb") as fp:
    pickle.dump({"p_diff": p_diff,
                 "p_shared": p_shared_locs_imgs,
                 "p_indp": p_indp_locs_imgs}, fp)